# 0.0 Setup e carregamento

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys
import importlib
warnings.filterwarnings('ignore')
sys.path.append('../src')
from feature_engineering import calc_woe_iv

# Carregar dados limpos
df = pd.read_csv('../data/processed_data/df_cleaned.csv')
target_col = 'inadipl_90dias_ult2anos'

print("=" * 80)
print("🔧 FEATURE ENGINEERING - PARTE 1")
print("=" * 80)
print(f" Dataset inicial:")
print(f"   Shape: {df.shape}")
print(f"   Features: {df.shape[1] - 1} (excluindo target)")

# Criar cópia para não alterar original
df_fe = df.copy()

🔧 FEATURE ENGINEERING - PARTE 1
 Dataset inicial:
   Shape: (149234, 21)
   Features: 20 (excluindo target)


In [62]:
df_fe.columns

Index(['inadipl_90dias_ult2anos', 'utilizacao_credito', 'idade',
       'atrasos_30dias', 'divida_ratio', 'renda_mensal',
       'linhas_credito_abertas', 'atrasos_90dias', 'emprestimos_imobiliarioss',
       'dependentes', 'comprometimento_renda', 'faixa_etaria',
       'renda_mensal_missing', 'comprometimento_renda_missing',
       'faixa_etaria_missing', 'utilizacao_credito_missing',
       'divida_ratio_missing', 'idade_outlier_flag',
       'comprometimento_outlier_flag', 'renda_outlier_flag',
       'utilizacao_outlier_flag'],
      dtype='str')

## Preparação dos dados

In [63]:
def faixa_to_num(faixa):
    try:
        # Se for apenas número (ex: "60"), converte direto
        if faixa.isdigit():
            return int(faixa)
        
        # Se for faixa com hífen (ex: "36-45")
        if "-" in faixa:
            limites = faixa.split("-")
            return (int(limites[0]) + int(limites[1])) / 2
        
        # Se for "65+" ou similar
        if "+" in faixa:
            return int(faixa.replace("+", ""))  # ex: "60+" -> 60
        
        # Se for "menos de 18"
        if "18" in faixa:
            return 18
        
        return None
    except:
        return None



In [64]:

# Carregar versões salvas
df_model_A = pd.read_csv("../data/df_model_A.csv")
df_model_B = pd.read_csv("../data/df_model_B.csv")

faixa_map = {
    "18-25": 1,
    "26-35": 2,
    "36-45": 3,
    "46-55": 4,
    "56+": 5
}
df_model_A["faixa_etaria_num"] = df_model_A["faixa_etaria"].map(faixa_map)
df_model_B["faixa_etaria_num"] = df_model_B["faixa_etaria"].map(faixa_map)




# 1.0 Derivation feature "Comprometimento_renda"

Essa variavel perdeu poder discriminatório após a limpeza de truncagem em 300%, os clientes ficaram num intervalo plausível, removendo distorçoes de outliers que inflavam os adimplentes e inadimplentes mas isso acabou "achatando" a variabilidade e fez o information valuate cair para 0.002 não conseguindo diferenciar risco de forma significativa.
Será usado o IV calculado com a métrica WOE para avaliar se a variável ganha força quando categorizada em faixas(bins) e revelar padrões escondidos. Isso ajuda a validar se vale a pena mantê-la no Projeto.

In [65]:
# função para calcular WOE e IV
#df_fe['target'] = 1 inadimp, 2 adimp.

# Definição dos bins 
bins = [0, 50, 100, 150, 200, 250, 300]
bins_faixa_etaria = [18,25,35,45,55,65,100]
# Chamada correta
woe_result, iv_value = calc_woe_iv(
    df_fe,
    'comprometimento_renda',
    'inadipl_90dias_ult2anos',
    bins=bins
)

print(woe_result)
print("IV total:", iv_value)


              bin   count   sum  non_event  event_rate       woe        iv
0  (-0.001, 50.0]   10914   579      10335    0.053051  0.249275  0.004083
1   (50.0, 100.0]    2824   159       2665    0.056303  0.186341  0.000606
2  (100.0, 150.0]    2178   209       1969    0.095960 -0.389767  0.002627
3  (150.0, 200.0]    1302   114       1188    0.087558 -0.288886  0.000826
4  (200.0, 250.0]    1331    99       1232    0.074380 -0.111439  0.000116
5  (250.0, 300.0]  130685  8848     121837    0.067705 -0.010221  0.000092
IV total: 0.008350182879276602


O resultado mostra que o IV total é de 0.0083 é extremamente baixo.
Ou seja, não tem poder de separar entre inadimplentes e adimplentes.
Isso acontece porque mais de 130 mil desses dados estão na faixa/bins de 250 a 300 e não traz variabilidade para diferenciar risco.
Os Bins abaixo de 200 mostra diferenças mas é uma fração pequena da base, então o modelo acaba vendo quase todo mundo igual.
* Talvez separar esses dados acima de 250 como categoria especial para não perder info.
* Ou criar variaveis derivadas para introduzir novas proporções para que não fique concentradas nos Bins mais altos.

In [66]:
# 1. Marcar os casos "não informado" na flag
df.loc[df["faixa_etaria"].str.lower() == "não informado", "faixa_etaria_missing"] = 1

# Converter para numérico, forçando strings inválidas a virarem NaN
df["faixa_etaria_num"] = pd.to_numeric(df["faixa_etaria"], errors="coerce")
bins_faixa_etaria = [18, 25, 35, 45, 55, 65, 100]


# Salvando dataframe com as variáveis já transformadas em WOE para modelagem
def apply_woe(df, var_name, woe_table, bins):
    """ df: dataframe original var_name: nome da variável original 
    woe_table: resultado da função calc_woe_iv (com colunas woe) 
    bins: lista/array de cortes usados no calc_woe_iv """ 
    # Criar categorias com os mesmos bins 
    categories = pd.cut(df[var_name], bins=bins, include_lowest=True)
    
    # Criar dicionário de mapeamento: intervalo -> WOE 
    woe_dict = dict(zip(woe_table['bin'], woe_table['woe']))     
    
    # Mapear cada categoria para o WOE 
    df[var_name + "_woe"] = categories.map(lambda x: woe_dict.get(x, None)) 
    return df


# Exemplo para uma variável
woe_result, iv_value = calc_woe_iv(df, 'comprometimento_renda', 'inadipl_90dias_ult2anos', bins=bins)
woe_result_etaria, iv_etaria = calc_woe_iv(
    df.dropna(subset=["faixa_etaria_num"]), 
    "faixa_etaria_num", 
    "inadipl_90dias_ult2anos", 
    bins=bins_faixa_etaria
)

# Salvar tabela WOE/IV
woe_result.to_csv("../outputs/woe_iv/woe_comprometimento_renda.csv", index=False)
woe_result_etaria.to_csv("../outputs/woe_iv/woe_faixa_etaria.csv", index=False)

# Criar df_woe com variáveis transformadas
df_woe = df.copy()
df_woe = apply_woe(df, "comprometimento_renda", woe_result, bins)
df_woe = apply_woe(df, "faixa_etaria_num", woe_result_etaria, bins_faixa_etaria)
print(woe_result_etaria)
print(df_woe["faixa_etaria_num_woe"].value_counts(dropna=False))

# Salvar dataset transformado
df_woe.to_pickle("../outputs/woe_iv/df_woe.pkl")


            bin  count   sum  non_event  event_rate  woe   iv
0  (55.0, 65.0]  44074  1336      42738    0.030313  0.0  0.0
faixa_etaria_num_woe
NaN    105160
0.0     44074
Name: count, dtype: int64


esses 105.160 NaN em faixa_etaria_num_woe não são um problema. Eles representam os clientes que não informaram idade.

Por que não é problema?
O WOE foi calculado apenas para quem tem idade válida (sem faixa etária)

Quem não informou idade não pode ser encaixado em nenhum bin de idade, então naturalmente fica como NaN.

Esse grupo é capturado pela variável faixa_etaria_missing, que você já criou.

No modelo final, você usa duas variáveis distintas:

faixa_etaria_num_woe → risco relativo das faixas válidas.

faixa_etaria_missing → risco específico de quem não informou.

Ou seja, o modelo continua cobrindo 100% da base:

~44 mil registros com idade informada → entram via faixa_etaria_num_woe.

~105 mil registros sem idade informada → entram via faixa_etaria_missing.


In [67]:
#----------------VARIAVEIS DERIVADAS-----------------------------
## 1. Utilização média por linha de crédito pessoal.
#Captura se o cliente concentra divídas em poucas linhas ou distribui melhor.
df_fe['utilizacao_media_linha'] = df_fe['utilizacao_credito'] / (df_fe['linhas_credito_abertas'] + 1e-6)

# 2. Comprometimento ajustado pela renda
df_fe['comprometimento_renda_ajustado'] = df_fe['divida_ratio'] / (df_fe['renda_mensal'] + 1e-6)


# 3. Índice de severidade de atrasos
df_fe['indice_severidade_atrasos'] = ( df_fe['atrasos_30dias']*1 + df_fe['atrasos_90dias']*3 )


# 4. Intensidade de crédito por idade
df_fe['intensidade_credito_idade'] = df_fe['linhas_credito_abertas'] / (df_fe['idade'] + 1e-6)


# 5. Proporção de crédito imobiliário
df_fe['proporcao_credito_imobiliario'] = df_fe['emprestimos_imobiliarioss'] / (df_fe['linhas_credito_abertas'] + 1e-6)


# 6. Renda per capita
df_fe['renda_per_capita'] = df_fe['renda_mensal'] / (df_fe['dependentes'] + 1)



In [68]:
# Lista de variáveis derivadas
variaveis_derivadas = [
    'utilizacao_media_linha',
    'comprometimento_renda_ajustado',
    'indice_severidade_atrasos',
    'intensidade_credito_idade',
    'proporcao_credito_imobiliario',
    'renda_per_capita'
]

# Definição de bins (ajuste conforme cada variável)
bins_dict = {
    'utilizacao_media_linha': [0, 0.25, 0.5, 1, 2, 5, 10],
    'comprometimento_renda_ajustado': [0, 1e-05, 5e-05, 1e-04, 5e-04, 1e-03],
    'indice_severidade_atrasos': [0, 1, 2, 3, 5, 10],
    'intensidade_credito_idade': [0, 0.1, 0.2, 0.5, 1, 2],
    'proporcao_credito_imobiliario': [0, 0.25, 0.5, 0.75, 1],
    'renda_per_capita': [0, 500, 1000, 2000, 5000, 10000, 20000]
}

# Loop para calcular IV de todas
iv_results = {}
for var in variaveis_derivadas:
    _, iv_value = calc_woe_iv(df_fe, var, 'inadipl_90dias_ult2anos', bins_dict[var])
    iv_results[var] = iv_value

# Converter em DataFrame para tabela comparativa
iv_table = pd.DataFrame.from_dict(iv_results, orient='index', columns=['IV'])
iv_table = iv_table.sort_values(by='IV', ascending=False)

print(iv_table)



                                      IV
indice_severidade_atrasos       0.981157
renda_per_capita                0.585426
utilizacao_media_linha          0.198515
comprometimento_renda_ajustado  0.088908
intensidade_credito_idade       0.022119
proporcao_credito_imobiliario   0.011419


In [69]:
df_fe[[
    'utilizacao_media_linha',
    'comprometimento_renda_ajustado',
    'indice_severidade_atrasos',
    'intensidade_credito_idade',
    'proporcao_credito_imobiliario',
    'renda_per_capita'
]]

,utilizacao_media_linha,comprometimento_renda_ajustado,indice_severidade_atrasos,intensidade_credito_idade,proporcao_credito_imobiliario,renda_per_capita
0,0.058931,8.804825e-05,2,0.288889,0.461538,3040.000000
1,0.239300,4.688462e-05,0,0.100000,0.000000,1300.000000
2,0.329100,2.797502e-05,4,0.052632,0.000000,3042.000000
3,0.046760,1.090909e-05,0,0.166667,0.000000,3300.000000
4,0.129600,4.980000e-07,1,0.142857,0.142857,50000.000000
...,...,...,...,...,...,...
149229,0.010175,1.071905e-04,0,0.054054,0.250000,2100.000000
149230,0.074925,1.283309e-04,0,0.090909,0.250000,1861.333333
149231,0.013667,6.151645e-01,0,0.310345,0.055556,6291.000000
149232,0.000000,0.000000e+00,0,0.133333,0.000000,5716.000000


In [70]:
# Lista de variáveis originais
variaveis_originais = [
    'utilizacao_credito',
    'idade',
    'atrasos_30dias',
    'divida_ratio',
    'renda_mensal',
    'linhas_credito_abertas',
    'atrasos_90dias',
    'emprestimos_imobiliarioss',
    'dependentes'
]

# Definição de bins para originais (ajustado conforme distribuição real)
bins_originais = {
    'utilizacao_credito': [0, 0.1, 0.3, 0.56, 0.9, 1.5, 2.0],
    'idade': [18, 25, 35, 45, 60, 80, 100],
    'atrasos_30dias': [0, 1, 2, 3, 5, 10],
    'divida_ratio': [0, 0.25, 0.5, 1, 2, 5],
    'renda_mensal': [0, 1000, 2000, 5000, 10000, 20000],
    'linhas_credito_abertas': [0, 2, 5, 10, 20, 50],
    'atrasos_90dias': [0, 1, 2, 3, 5, 10],
    'emprestimos_imobiliarioss': [0, 1, 2, 3, 5, 10,20,30],
    'dependentes': [0, 1, 2, 3, 5, 10]
}

# Loop para calcular IV das variáveis originais
iv_results_originais = {}
for var in variaveis_originais:
    # supondo que calc_woe_iv retorna (woe_table, iv_value)
    _, iv_value = calc_woe_iv(df_fe, var, 'inadipl_90dias_ult2anos', bins_originais[var])
    iv_results_originais[var] = iv_value

# Extrair apenas o IV das originais
iv_results_originais_iv = {var: iv_results_originais[var] for var in iv_results_originais}

# Para derivadas, se já são números, não precisa acessar índice
iv_results_iv = {var: iv_results[var] for var in iv_results}




In [71]:
print("=" * 80)
print("💰 CRIANDO FEATURES DE RAZÃO FINANCEIRA (COMPLEMENTO)")
print("=" * 80)

# --- BLOCO 1: Utilização de Crédito ---
# Equivalente a RevolvingUtilizationOfUnsecuredLines
if 'utilizacao_credito' in df_fe.columns:

    df_fe['utilizacao_credito_categoria'] = pd.cut(
        df_fe['utilizacao_credito'],
        bins=[0, 0.1, 0.3, 0.56, 0.9, 1.5, 2.0, float('inf')],
        labels=['Muito_Baixa', 'Baixa', 'Media', 'Alta', 'Muito_Alta', 'Critica', 'Extrema']
    )
    print("✅ utilizacao_credito_categoria criada")

    df_fe['alta_utilizacao_flag'] = (
        df_fe['utilizacao_credito'] > 0.9
    ).astype(int)
    print("✅ alta_utilizacao_flag criada")

# --- BLOCO 2: Razão Dívida/Renda ---
# Equivalente a DebtRatio + MonthlyIncome
if 'renda_mensal' in df_fe.columns and 'divida_ratio' in df_fe.columns:

    # Dívida absoluta mensal
    df_fe['total_renda'] = df_fe['renda_mensal'] * df_fe['divida_ratio']
    print("✅ total_renda criada")

    # Classificação por nível de endividamento
    df_fe['renda_nivel'] = pd.cut(
        df_fe['divida_ratio'],
        bins=[0, 0.2, 0.4, 0.6, float('inf')],
        labels=['Baixo', 'Moderado', 'Alto', 'Critico']
    )
    print("✅ renda_nivel criada")

    # Renda disponível após dívidas
    df_fe['renda_disponivel'] = df_fe['renda_mensal'] * (1 - df_fe['divida_ratio'])
    df_fe['renda_disponivel'] = df_fe['renda_disponivel'].clip(lower=0)
    print("✅ renda_disponivel criada")

# --- BLOCO 3: Flag de renda per capita baixa ---
# renda_per_capita já existe, só cria a flag
if 'renda_per_capita' in df_fe.columns:

    df_fe['baixa_renda_por_pessoa'] = (
        df_fe['renda_per_capita'] < 1412
    ).astype(int)
    print("✅ baixa_renda_por_pessoa criada")

print(f"\n📊 Features após complemento de razões financeiras: {df_fe.shape[1]}")

💰 CRIANDO FEATURES DE RAZÃO FINANCEIRA (COMPLEMENTO)
✅ utilizacao_credito_categoria criada
✅ alta_utilizacao_flag criada
✅ total_renda criada
✅ renda_nivel criada
✅ renda_disponivel criada
✅ baixa_renda_por_pessoa criada

📊 Features após complemento de razões financeiras: 33


In [72]:
# features IDADE/TEMPO
df_fe['is_jovem']     = (df_fe['idade'] < 25).astype(int)
df_fe['is_idoso']     = (df_fe['idade'] >= 65).astype(int)
df_fe['idade_decada'] = (df_fe['idade'] // 10) * 10

In [73]:
# features histórico de crédito
delay_cols = [col for col in ['atrasos_30dias', 'atrasos_90dias'] 
              if col in df_fe.columns]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
if len(delay_cols) > 0:

    # Total de atrasos
    df_fe['total_atrasos'] = df_fe[delay_cols].sum(axis=1)
    print("✅ total_atrasos criada")

    # Flag de qualquer atraso
    df_fe['possui_atraso'] = (df_fe['total_atrasos'] > 0).astype(int)
    print("✅ possui_atraso criada")

    # Comportamento de pagamento
    conditions = [
        (df_fe['total_atrasos'] == 0),
        (df_fe['total_atrasos'] <= 2),
        (df_fe['total_atrasos'] <= 5),
        (df_fe['total_atrasos'] > 5)
    ]
    choices = ['Excelente', 'Bom', 'Regular', 'Ruim']
    
    df_fe['comportamento_pagamento'] = np.select(conditions, choices, default='Desconhecido')
    print("✅ comportamento_pagamento criada")


# Linhas de crédito abertas
if 'linhas_credito_abertas' in df_fe.columns:

    df_fe['linhas_credito_categoria'] = pd.cut(
        df_fe['linhas_credito_abertas'],
        bins=[0, 3, 7, 15, float('inf')],
        labels=['Poucas', 'Normal', 'Muitas', 'Excessivas'],
        include_lowest=True
    )
    print("✅ linhas_credito_categoria criada")

    df_fe['muitas_linhas_credito'] = (
        df_fe['linhas_credito_abertas'] > 10
    ).astype(int)
    print("✅ muitas_linhas_credito criada")

print(f"\n📊 Features após histórico de crédito: {df_fe.shape[1]}")

✅ total_atrasos criada
✅ possui_atraso criada
✅ comportamento_pagamento criada
✅ linhas_credito_categoria criada
✅ muitas_linhas_credito criada

📊 Features após histórico de crédito: 41


In [74]:
print("=" * 80)
print("🔗 CRIANDO FEATURES COMPOSTAS")
print("=" * 80)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# --- Feature 1: Score de Risco Combinado ---
if all(col in df_fe.columns for col in ['total_atrasos', 'divida_ratio']):

    df_fe['risco_score'] = (
        scaler.fit_transform(df_fe[['total_atrasos']]) * 0.6 +
        scaler.fit_transform(df_fe[['divida_ratio']]) * 0.4
    ).flatten()
    print("✅ risco_score criada")

    df_fe['risco_categoria'] = pd.cut(
        df_fe['risco_score'],
        bins=[0, 0.25, 0.5, 0.75, 1.0],
        labels=['Baixo', 'Medio', 'Alto', 'Muito_Alto'],
        include_lowest=True
    )
    print("✅ risco_categoria criada")

else:
    print(f"⚠️  Colunas faltando para risco_score: "
          f"{[c for c in ['total_atrasos','divida_ratio'] if c not in df_fe.columns]}")

# --- Feature 2: Estabilidade Financeira ---
if all(col in df_fe.columns for col in ['idade', 'renda_mensal']):

    df_fe['estabilidade_financeira'] = (
        (df_fe['idade'] > 30).astype(int) +
        (df_fe['renda_mensal'] > df_fe['renda_mensal'].median()).astype(int)
    )
    # 0 = baixa, 1 = média, 2 = alta estabilidade
    print("✅ estabilidade_financeira criada")

else:
    print(f"⚠️  Colunas faltando para estabilidade_financeira: "
          f"{[c for c in ['idade','renda_mensal'] if c not in df_fe.columns]}")

# --- Feature 3: Capacidade de Pagamento ---
if all(col in df_fe.columns for col in ['renda_disponivel', 'linhas_credito_abertas']):

    df_fe['capacidade_pagamento'] = (
        df_fe['renda_disponivel'] /
        (df_fe['linhas_credito_abertas'] + 1)
    )
    print("✅ capacidade_pagamento criada")

else:
    print(f"⚠️  Colunas faltando para capacidade_pagamento: "
          f"{[c for c in ['renda_disponivel','linhas_credito_abertas'] if c not in df_fe.columns]}")

print(f"\n📊 Features finais Parte 1: {df_fe.shape[1]}")

🔗 CRIANDO FEATURES COMPOSTAS
✅ risco_score criada
✅ risco_categoria criada
✅ estabilidade_financeira criada
✅ capacidade_pagamento criada

📊 Features finais Parte 1: 45


In [75]:
# ============================================================
# 🔗 IV — FEATURES COMPOSTAS
# ============================================================
# --- estabilidade_financeira ---
iv_compostas_binarias={}
iv_compostas_continuas={}
if 'estabilidade_financeira' in df_fe.columns:
    _, iv = calc_woe_iv(
        df_fe, 'estabilidade_financeira',
        'inadipl_90dias_ult2anos',
        bins=[-0.5, 0.5, 1.5, 2.5]
    )
    iv_compostas_binarias['estabilidade_financeira'] = iv
    print(f"✅ IV calculado: estabilidade_financeira = {iv:.4f}")

# --- capacidade_pagamento ---
if 'capacidade_pagamento' in df_fe.columns:
    _, iv = calc_woe_iv(
        df_fe, 'capacidade_pagamento',
        'inadipl_90dias_ult2anos',
        bins=[0, 500, 1000, 2000, 5000, 10000, 50000]
    )
    iv_compostas_continuas['capacidade_pagamento'] = iv
    print(f"✅ IV calculado: capacidade_pagamento = {iv:.4f}")


✅ IV calculado: estabilidade_financeira = 0.0547
✅ IV calculado: capacidade_pagamento = 0.0294


In [76]:
# ============================================================
# 📊 CALCULANDO IV DAS FEATURES COMPLEMENTARES
# ============================================================

# --- Features numéricas contínuas (precisam de bins) ---
novas_vars_continuas = {
    'total_renda':       [0, 500, 1000, 2000, 5000, 10000, 50000],
    'renda_disponivel': [0, 500, 1000, 2000, 5000, 10000, 50000],
    'renda_per_capita': [0, 500, 1000, 2000, 5000, 10000, 20000],  # já existe, recalcula
}

iv_novas_continuas = {}
for var, bins in novas_vars_continuas.items():
    if var in df_fe.columns:
        _, iv_value = calc_woe_iv(df_fe, var, 'inadipl_90dias_ult2anos', bins)
        iv_novas_continuas[var] = iv_value
        print(f"✅ IV calculado: {var} = {iv_value:.4f}")
    else:
        print(f"⚠️  Coluna não encontrada: {var}")

# --- Features binárias (0/1) — bins simples ---
novas_vars_binarias = {
    'alta_utilizacao_flag': [-0.5, 0.5, 1.5],
    'baixa_renda_por_pessoa': [-0.5, 0.5, 1.5],
}

iv_novas_binarias = {}
for var, bins in novas_vars_binarias.items():
    if var in df_fe.columns:
        _, iv_value = calc_woe_iv(df_fe, var, 'inadipl_90dias_ult2anos', bins)
        iv_novas_binarias[var] = iv_value
        print(f"✅ IV calculado: {var} = {iv_value:.4f}")
    else:
        print(f"⚠️  Coluna não encontrada: {var}")

# --- Features categóricas (encoding ordinal antes do IV) ---
# credit_utilization_category e debt_level precisam virar número
cat_map_utilizacao = {'Baixa': 1, 'Media': 2, 'Alta': 3, 'Muito_Alta': 4}
cat_map_debt       = {'Baixo': 1, 'Moderado': 2, 'Alto': 3, 'Critico': 4}

iv_novas_categoricas = {}

if 'utilizacao_credito_categoria ' in df_fe.columns:
    df_fe['utilizacao_credito_categoria _num'] = (
        df_fe['utilizacao_credito_categoria '].map(cat_map_utilizacao)
    )
    _, iv_value = calc_woe_iv(
        df_fe.dropna(subset=['utilizacao_credito_categoria _num']),
        'utilizacao_credito_categoria _num',
        'inadipl_90dias_ult2anos',
        bins=[0.5, 1.5, 2.5, 3.5, 4.5]
    )
    iv_novas_categoricas['utilizacao_credito_categoria '] = iv_value
    print(f"✅ IV calculado: utilizacao_credito_categoria = {iv_value:.4f}")

if 'renda_nivel' in df_fe.columns:
    df_fe['renda_nivel_num'] = df_fe['renda_nivel'].map(cat_map_debt)
    _, iv_value = calc_woe_iv(
        df_fe.dropna(subset=['renda_nivel_num']),
        'renda_nivel_num',
        'inadipl_90dias_ult2anos',
        bins=[0.5, 1.5, 2.5, 3.5, 4.5]
    )
    iv_novas_categoricas['renda_nivel'] = iv_value
    print(f"✅ IV calculado: debt_level = {iv_value:.4f}")

# ============================================================
# 📊 TABELA CONSOLIDADA — TODAS AS FEATURES (ORIGINAIS + DERIVADAS + NOVAS)
# ============================================================

iv_novas_todas = {**iv_novas_continuas, **iv_novas_binarias, **iv_novas_categoricas,**iv_compostas_binarias,**iv_compostas_binarias,**iv_compostas_continuas}

iv_novas_df = pd.DataFrame.from_dict(iv_novas_todas, orient='index', columns=['IV'])
iv_novas_df = iv_novas_df.sort_values('IV', ascending=False)

print("\n" + "=" * 50)
print("📊 IV DAS FEATURES COMPLEMENTARES")
print("=" * 50)
print(iv_novas_df.to_string())

# Referência de interpretação
print("""
📖 Referência de IV:
   < 0.02  → Sem poder preditivo
   0.02–0.1 → Fraco
   0.1–0.3  → Médio
   0.3–0.5  → Forte
   > 0.5    → Suspeito (possível data leakage)
""")

✅ IV calculado: total_renda = 0.0172
✅ IV calculado: renda_disponivel = 0.0909
✅ IV calculado: renda_per_capita = 0.5854
✅ IV calculado: alta_utilizacao_flag = 0.6342
✅ IV calculado: baixa_renda_por_pessoa = 0.4283
✅ IV calculado: debt_level = 0.0226

📊 IV DAS FEATURES COMPLEMENTARES
                               IV
alta_utilizacao_flag     0.634193
renda_per_capita         0.585426
baixa_renda_por_pessoa   0.428317
renda_disponivel         0.090915
estabilidade_financeira  0.054676
capacidade_pagamento     0.029360
renda_nivel              0.022572
total_renda              0.017244

📖 Referência de IV:
   < 0.02  → Sem poder preditivo
   0.02–0.1 → Fraco
   0.1–0.3  → Médio
   0.3–0.5  → Forte
   > 0.5    → Suspeito (possível data leakage)



In [77]:
# Juntar originais e derivadas
iv_novas_dict = iv_novas_df['IV'].to_dict()
iv_results_total = {**iv_results_originais_iv, **iv_results_iv,**iv_novas_dict}

# Converter em DataFrame
iv_table_total = pd.DataFrame.from_dict(iv_results_total, orient='index', columns=['IV'])
iv_table_total = iv_table_total.sort_values(by='IV', ascending=False)

print("\n" + "=" * 50)
print("🏆 RANKING COMPLETO DE TODAS AS FEATURES")
print("=" * 50)
print(iv_table_total.to_string())


🏆 RANKING COMPLETO DE TODAS AS FEATURES
                                      IV
utilizacao_credito              1.093495
indice_severidade_atrasos       0.981157
alta_utilizacao_flag            0.634193
renda_per_capita                0.585426
atrasos_30dias                  0.455271
atrasos_90dias                  0.454149
baixa_renda_por_pessoa          0.428317
idade                           0.229179
dependentes                     0.213090
utilizacao_media_linha          0.198515
renda_disponivel                0.090915
comprometimento_renda_ajustado  0.088908
renda_mensal                    0.085137
linhas_credito_abertas          0.084017
divida_ratio                    0.068745
estabilidade_financeira         0.054676
capacidade_pagamento            0.029360
renda_nivel                     0.022572
intensidade_credito_idade       0.022119
emprestimos_imobiliarioss       0.022058
total_renda                     0.017244
proporcao_credito_imobiliario   0.011419


In [78]:
# Definir limiar de IV (ajuste conforme necessidade)
iv_threshold = 0.05

# Selecionar variáveis com IV acima do limiar
selected_vars = iv_table_total[iv_table_total['IV'] > iv_threshold].index.tolist()

print("Variáveis selecionadas:", selected_vars)

# Criar dataset filtrado apenas com as variáveis selecionadas
df_features_selected = df_fe[selected_vars]

print("Shape do dataset filtrado:", df_features_selected.shape)

Variáveis selecionadas: ['utilizacao_credito', 'indice_severidade_atrasos', 'alta_utilizacao_flag', 'renda_per_capita', 'atrasos_30dias', 'atrasos_90dias', 'baixa_renda_por_pessoa', 'idade', 'dependentes', 'utilizacao_media_linha', 'renda_disponivel', 'comprometimento_renda_ajustado', 'renda_mensal', 'linhas_credito_abertas', 'divida_ratio', 'estabilidade_financeira']
Shape do dataset filtrado: (149234, 16)


In [82]:
# Incluindo a coluna target ao dataset
df_features_selected["inadipl_90dias_ult2anos"] = df_fe["inadipl_90dias_ult2anos"]

df_features_selected["inadipl_90dias_ult2anos"]

0          True
1         False
2         False
3         False
4         False
          ...  
149229    False
149230    False
149231    False
149232    False
149233    False
Name: inadipl_90dias_ult2anos, Length: 149234, dtype: bool

In [83]:
# Transformando em WOE calculado
woe_map = {
    "Excelente": 0.45,
    "Bom": 0.12,
    "Regular": -0.08,
    "Ruim": -0.32
}
df_fe["comportamento_pagamento_woe"] = df_fe["comportamento_pagamento"].map(woe_map)
# Salvar versões A e B já com faixa_etaria_num
df_model_A.to_csv("../data/df_model_A_fe.csv", index=False)
df_model_B.to_csv("../data/df_model_B_fe.csv", index=False)

In [84]:
OUTPUT_FEAT = "../data/features"
# Colunas criadas no 03_data_cleaning que têm valor preditivo
# mas não estavam no df_fe quando o IV foi calculado
COLS_CLEANING = [
    # Flags de missing
    'renda_mensal_missing',
    'comprometimento_renda_missing',
    'faixa_etaria_missing',
    'utilizacao_credito_missing',
    'divida_ratio_missing',
    # Flags de outlier
    'comprometimento_outlier_flag',
    'idade_outlier_flag',
    'renda_outlier_flag',
    'utilizacao_outlier_flag',
    # Variáveis originais com poder preditivo
    'emprestimos_imobiliarioss',
    'faixa_etaria_num',
]

# Verificar quais existem no df_fe (foram carregadas do df_cleaned.csv)
cols_disponiveis = [c for c in COLS_CLEANING if c in df_fe.columns]
cols_faltando    = [c for c in COLS_CLEANING if c not in df_fe.columns]

print(f"Colunas disponíveis para adicionar : {cols_disponiveis}")
print(f"Colunas não encontradas no df_fe   : {cols_faltando}")

# Adicionar ao df_features_selected antes de exportar
for col in cols_disponiveis:
    if col not in df_features_selected.columns:
        df_features_selected[col] = df_fe[col]
        print(f"✅ {col} adicionada")

# Re-exportar
df_features_selected.to_csv(
    os.path.join(OUTPUT_FEAT, 'df_features_selected.csv'),
    index=False, encoding='utf-8-sig'
)
print(f"\n✅ df_features_selected re-exportado com {df_features_selected.shape[1]} colunas")

Colunas disponíveis para adicionar : ['renda_mensal_missing', 'comprometimento_renda_missing', 'faixa_etaria_missing', 'utilizacao_credito_missing', 'divida_ratio_missing', 'comprometimento_outlier_flag', 'idade_outlier_flag', 'renda_outlier_flag', 'utilizacao_outlier_flag', 'emprestimos_imobiliarioss']
Colunas não encontradas no df_fe   : ['faixa_etaria_num']
✅ renda_mensal_missing adicionada
✅ comprometimento_renda_missing adicionada
✅ faixa_etaria_missing adicionada
✅ utilizacao_credito_missing adicionada
✅ divida_ratio_missing adicionada
✅ comprometimento_outlier_flag adicionada
✅ idade_outlier_flag adicionada
✅ renda_outlier_flag adicionada
✅ utilizacao_outlier_flag adicionada
✅ emprestimos_imobiliarioss adicionada

✅ df_features_selected re-exportado com 27 colunas


In [85]:
#salvando em ..data/features
caminho_dir = r"C:\Users\wesle\anaconda3\envs\Credit_Score_Project\data\features"
os.makedirs(caminho_dir, exist_ok=True)
df_features_selected.to_csv(os.path.join(caminho_dir,"df_features_selected.csv"), index=False,encoding="utf-8-sig")

In [87]:
df_features_selected

,utilizacao_credito,indice_severidade_atrasos,alta_utilizacao_flag,renda_per_capita,atrasos_30dias,atrasos_90dias,baixa_renda_por_pessoa,idade,dependentes,utilizacao_media_linha,...,renda_mensal_missing,comprometimento_renda_missing,faixa_etaria_missing,utilizacao_credito_missing,divida_ratio_missing,comprometimento_outlier_flag,idade_outlier_flag,renda_outlier_flag,utilizacao_outlier_flag,emprestimos_imobiliarioss
0,0.7661,2,0,3040.000000,2,0,0,45,2,0.058931,...,0,0,0,0,0,1,0,0,0,6
1,0.9572,0,1,1300.000000,0,0,1,40,1,0.239300,...,0,0,0,0,0,1,0,0,0,0
2,0.6582,4,0,3042.000000,1,1,0,38,0,0.329100,...,0,0,0,0,0,1,0,0,0,0
3,0.2338,0,0,3300.000000,0,0,0,30,0,0.046760,...,0,0,0,0,0,1,0,0,0,0
4,0.9072,1,1,50000.000000,1,0,0,49,0,0.129600,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149229,0.0407,0,0,2100.000000,0,0,0,74,0,0.010175,...,0,0,0,0,0,1,0,0,0,1
149230,0.2997,0,0,1861.333333,0,0,0,44,2,0.074925,...,0,0,0,0,0,1,0,0,0,1
149231,0.2460,0,0,6291.000000,0,0,0,58,0,0.013667,...,1,1,0,0,0,0,0,0,0,1
149232,0.0000,0,0,5716.000000,0,0,0,30,0,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_woe.to_pickle("../outputs/woe_iv/df_woe.pkl")
